In [2]:
import pygame
import numpy as np
import tensorflow as tf
import time
import os
from Core_Game_Parts import *
from keras import layers, models


In [3]:
def build_dueling_dqn(input_shape=(4,), action_size=3):
    from keras import layers, models, backend as K
    import tensorflow as tf

    inputs = layers.Input(shape=input_shape)
    x = layers.Dense(128, activation='relu')(inputs)
    x = layers.Dense(128, activation='relu')(x)

    # Dueling streams
    value = layers.Dense(64, activation='relu')(x)
    value = layers.Dense(1, activation='linear')(value)

    advantage = layers.Dense(64, activation='relu')(x)
    advantage = layers.Dense(action_size, activation='linear')(advantage)

    # Correct combination using Lambda layers to maintain graph compatibility
    def combine_dueling(inputs):
        value, advantage = inputs
        return value + (advantage - tf.reduce_mean(advantage, axis=1, keepdims=True))

    q_values = layers.Lambda(combine_dueling)([value, advantage])

    model = models.Model(inputs=inputs, outputs=q_values)
    return model


In [4]:


def run_d3qn_agent(weights_path, render=True):
    """
    Visual simulation for the trained D3QN agent (Double + Dueling).
    """

    # Use proper SDL backend
    os.environ["SDL_VIDEODRIVER"] = "windows"
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    clock = pygame.time.Clock()
    pygame.display.set_caption("Trained D3QN Agent")
    font = pygame.font.SysFont(None, 36)
    crash_font = pygame.font.SysFont(None, 72)

    # Load the environment
    track_surface = pygame.image.load(TRACK_IMAGE_PATH).convert()

    # Build model and load weights
    ai_model = build_dueling_dqn()
    try:
        ai_model.load_weights(weights_path)
        print(f"Successfully loaded D3QN weights from: {weights_path}")
    except Exception as e:
        print(f"Error loading weights: {e}")
        pygame.quit()
        return

    car = Car(CAR_IMAGE_PATH, 900, 426, angle=-45)
    running = True
    action_counts = {0: 0, 1: 0, 2: 0}  # Left / Straight / Right
    step_counter = 0

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # 1️⃣ Build state vector (3 sensors + speed)
        distances, ray_endpoints = ray_casting(car, track_surface)
        state = np.array(distances + [car.speed], dtype=np.float32)
        state = np.expand_dims(state, axis=0)

        # 2️⃣ Predict Q-values and select best action
        q_values = ai_model.predict(state, verbose=0)
        action = np.argmax(q_values[0])
        action_counts[action] += 1

        # 3️⃣ Execute action
        if action == 0:
            car.angle += 5
        elif action == 2:
            car.angle -= 5
        car.speed = min(car.speed + 0.05, 5)
        car.move()

        # 4️⃣ Check for crash (off-track color)
        crashed = False
        try:
            pixel_color = track_surface.get_at((int(car.x), int(car.y)))[:3]
            if pixel_color == (50, 50, 50):  # your DRAW_COLOR
                crashed = True
        except:
            crashed = True

        if crashed:
            # Display crash message
            crash_text = crash_font.render("CRASHED! Resetting...", True, (255, 0, 0))
            text_rect = crash_text.get_rect(center=(SCREEN_WIDTH/2, SCREEN_HEIGHT/2))
            screen.blit(crash_text, text_rect)
            pygame.display.update()
            pygame.time.wait(2000)
            print(f"Episode crash after {step_counter} steps.")
            print("Action counts:", action_counts)
            # Reset car
            car = Car(CAR_IMAGE_PATH, 900, 426, angle=-45)
            car.speed = 0
            action_counts = {0: 0, 1: 0, 2: 0}
            step_counter = 0

        # 5️⃣ Render everything
        if render:
            screen.blit(track_surface, (0, 0))
            car.draw(screen)
            for p in ray_endpoints:
                pygame.draw.line(screen, (0, 255, 0), (car.x, car.y), p, 1)

            action_map = {0: "Left", 1: "Straight", 2: "Right"}
            info_text = f"Action: {action_map[action]} | Speed: {car.speed:.2f}"
            text_surface = font.render(info_text, True, (255, 255, 255))
            screen.blit(text_surface, (20, 20))

            pygame.display.update()
            clock.tick(60)

        step_counter += 1

    pygame.quit()


In [5]:
WEIGHTS_FILENAME = "best_d3qn_36checkpoints.weights.h5"   # or whatever you named it d3qn.weights.h5

if not pygame.display.get_init():
    run_d3qn_agent(WEIGHTS_FILENAME, render=True)
else:
    print("Pygame window may already be open. Restart the kernel and try again.")  # 1 19


Successfully loaded D3QN weights from: best_d3qn_36checkpoints.weights.h5
Episode crash after 414 steps.
Action counts: {0: 75, 1: 211, 2: 129}
Episode crash after 415 steps.
Action counts: {0: 75, 1: 211, 2: 129}
Episode crash after 415 steps.
Action counts: {0: 75, 1: 211, 2: 129}
